# Lifting State Up in React 

## Introduction

This lecture explains the concept of **lifting state up** in React, an important pattern for interviews and real-world applications where multiple components need to share and synchronize the same state. 
It builds on prior knowledge of **components**, **props**, and the **useState hook**, and shows how data can effectively flow from child to parent and between sibling components. 

***

## Props and Component Tree

Props are the mechanism by which a **parent component passes data to its child components** in React. 
In a component tree, data via props naturally flows **one way: from parent to child**, not directly from child to parent or between siblings. 

Key points:

- A component tree can be visualized as:
  - One parent component
  - Multiple child components
  - Sibling components sharing the same parent 
- Data transfer with props:
  - Parent → Child A (via props)
  - Parent → Child B (via props)
  - No direct Child A → Child B prop passing 

Important term:

- **Props**: Read-only data passed from a parent to a child component, used by the child for rendering. 

***

## The Need for Lifting State Up

Two main scenarios create the need for lifting state up:

1. **Child to Parent Communication**  
   - Requirement: Data is generated or changed in a child component but must be **known or used by the parent**. 
   - Example: A child input component updates a value that the parent must display or process. 

2. **Sharing State Between Siblings**  
   - Scenario:  
     - Child A creates/maintains some state.  
     - Child B also needs access to the **same state**, and both should remain **in sync**. 
   - Direct transfer from Child A to Child B is not possible via props because props only go from parent to child. 

Conclusion:  
To synchronize state between multiple components (especially siblings) or to reflect child-originated changes at the parent level, the state must be **moved (lifted) from the child component to their common parent**. 

***

## Core Idea of Lifting State Up

**Definition:**  
Lifting state up means **moving a piece of state and its update logic from a lower-level component (child) to a higher-level common ancestor (parent)** so that multiple components can share and stay synchronized with that state. 

Core principles:

- The **parent**:
  - Creates the state using `useState`. 
  - Manages and changes the state. 
  - Passes:
    - The state value itself.
    - The state updater function (setter) to children via props. 

- The **children**:
  - Receive the state value and setter function via props. 
  - Use the state value for display.
  - Call the setter function to request changes to the state. 

Effect:

- Any change triggered from **any child** updates the **single source of truth** in the parent.  
- All children receiving that state get the **updated value**, ensuring perfect synchronization. 

***

## Basic Parent-to-Child Props Example

Before lifting state, the lecture revisits the basic pattern:

- `App` component (parent) uses a `Card` component (child). 
- Parent passes a value via props:
  - Example: `<Card name="Love Babbar" />`. 
- Child reads props:
  - Example: displays `props.name` in JSX. 

Key points:

- This is **simple one-way data flow**: parent → child. 
- No state lifting is involved yet; this is the normal props usage pattern. 

***

## Converting Child State to Parent State

### Original (Not Lifted) Approach

- Child component maintains its own state:
  - Example: `Card` has `const [name, setName] = useState("")` inside it. 
  - Child’s input field updates `name` locally. 

Issues:

- Parent has **no access** to this `name` state.
- Sibling components **cannot share** this state.
- State is **isolated** inside one child component. 

### Lifting the State to the Parent

To enable sharing and parent awareness:

1. **Move state to the parent**  
   - In `App` (parent):
     - `const [name, setName] = useState("");`. 
   - The parent now owns the state.

2. **Pass state and setter to children via props**  
   - Example:  
     - `<Card name={name} setName={setName} />`. 
   - Every child that should share the state receives **both**:
     - `name` (state value)
     - `setName` (state updater function) 

3. **Child uses props instead of local state**  
   - Child uses:
     - `props.name` to display the value. 
     - `props.setName` to update it. 

Result:

- State is centralized in the parent.
- Child is now **controlled** by parent’s state.
- Multiple components can share and reflect this same state. 

***

## Implementing Child-to-Parent Updates

Once state is in the parent, child still appears to “update” it by calling the setter function passed down.

Typical pattern inside Child:

- Input field:
  - `<input type="text" onChange={handleChange} />`. 
- Handler:
  - `function handleChange(e) { props.setName(e.target.value); }`. 

What happens:

- User types in the child’s input.
- `onChange` triggers with event `e`.
- Child calls `props.setName(e.target.value)`.
- Parent’s `name` state updates.
- All components receiving `name` via props get the new value and re-render. 

Verification:

- Child component prints: `Name state variable inside Card: {props.name}`. 
- Parent component prints: `I am inside parent component and value of name is {name}`. 
- Typing in the child input updates both parent and child displays simultaneously, confirming lifting is working. 

***

## Synchronizing Sibling Components with Lifting State

Once state is in the parent, adding more synchronized children is straightforward.

### Example: Two Card Components

In parent:

- `const [name, setName] = useState("");`. 
- Two children:
  - `<Card title="Card One" name={name} setName={setName} />`  
  - `<Card title="Card Two" name={name} setName={setName} />`. 

In child (`Card`):

- Reads and displays:
  - `props.title`. 
  - `props.name`. 
- Uses `props.setName` in the input change handler. 

Effect:

- Typing into input (even in one card):
  - Updates parent’s `name`.
  - Reflects updated `name` in **both Card One and Card Two**. 
- All sibling components that receive the same state mirror the same value, staying perfectly synchronized. 

Key takeaway:

- **Any number of children** can share the same state by receiving the same parent-owned state and setter via props. 

***

## Conceptual Summary of Lifting State

Conceptual model:

- Parent component
  - Children: Child A, Child B, Child C, etc. 
- Originally:
  - Child A has its own `stateA`.
  - Child B has its own `stateB`.
  - They are independent and cannot stay in sync easily. 

With lifting state:

- Move shared state to parent:
  - Parent defines a single `stateShared` and its setter. 
- Parent passes `stateShared` and setter to:
  - Child A via props.
  - Child B via props.
  - Other children if needed. 
- Child A and Child B now both:
  - Use `stateShared` for rendering.
  - Use the same setter function to update it. 

Result:

- All children now depend on **one centralized state**.
- They behave in a synchronized, predictable way.
- Adding more children that share the same state is easy. 

***

## Practical Use Cases from React Docs

The lecture references official React documentation examples that demonstrate lifting state up in real UIs. 

### Example 1: Mutually Exclusive Panels (About vs. Emoji)

Scenario:

- There are two sibling panels, e.g.:
  - Panel About
  - Panel Emoji 
- Requirement:
  - When About is shown, Emoji should be hidden.
  - When Emoji is shown, About should be hidden. 

Problem without lifting:

- If each panel maintains its own local “open/closed” state, both can be open at the same time, causing inconsistent UI. 

Solution with lifting:

- Move the “which panel is open” state to their common parent. 
- Parent:
  - Holds state indicating which panel is active.
  - Passes down:
    - Active state info.
    - A function to change active panel. 
- Each panel:
  - Uses props to know whether it should render itself.
  - Calls the provided function to request state changes when clicked. 

Effect:

- When About is active:
  - Emoji hides.
- When Emoji becomes active:
  - About hides.
- Panels remain strictly mutually exclusive and fully synced. 

***

## Example 2: Synced Inputs

Scenario:

- Two input fields that should always show the **same text**:
  - Typing in the first should update the second.
  - Typing in the second should update the first. 

Problem without lifting:

- If each input has its own local state, changes in one do not reflect in the other. 

Solution with lifting:

- State is moved to the parent:
  - Parent defines a single text state. 
- Both inputs receive:
  - The same state value.
  - The same state-updating function via props. 

Effect:

- Typing in input 1:
  - Parent state updates.
  - Input 2 receives new value via props and updates its display. 
- Typing in input 2:
  - Same process in reverse.
- Both inputs remain perfectly synchronized. 

***

## Example 3: Filtering a List

Scenario:

- A search bar filters a list of items as the user types. 

Problem without lifting:

- If the filter text is maintained only in the search bar component, the list component may not get updated filtering criteria. 

Solution with lifting:

- Parent component holds:
  - `filterText` as state.
  - The complete list of items. 
- Parent passes:
  - `filterText` and its setter to the search bar.
  - Filtered list (derived from `filterText`) or `filterText` itself to the list component. 

Effect:

- When the user types in the search bar:
  - Parent `filterText` updates.
  - List re-renders with items filtered based on updated `filterText`. 
- Search UI and list stay in sync with one shared state. 

***

## When to Lift State Up

Lifting state up is useful when:

- Multiple components need to **share and stay synchronized** on the same piece of data. 
- A **child’s changes must be visible or processed in a parent**. 
- Independent local states cause:
  - Duplicate logic.
  - Inconsistent UI.
  - Bugs due to desynchronized values. 

General guideline:

- If two or more components need access to the same changing data, consider **moving that data and its updater to their closest common ancestor**. 

***

## Summary of Main Takeaways

- **Props flow one-way**: from parent to child; there is no built-in direct child-to-parent or sibling-to-sibling data flow. 
- **Lifting state up** means moving shared or important state from child components to their common parent so that the parent becomes the single source of truth. 
- The parent **creates, manages, and changes** the state using `useState` and passes:
  - The state value.
  - The state setter function to children via props. 
- Children use the received value for display and call the received setter function to request updates, enabling child-to-parent updates and sibling synchronization. 
- This pattern is essential for:
  - Syncing sibling components (e.g., multiple cards, synced inputs).
  - Controlling complex UIs (e.g., mutually exclusive panels, filtered lists).
  - Avoiding inconsistent state spread across multiple components.